<br>

## 데이터 읽어오기

<br>

In [4]:
"""
glob 모듈 사용하여 모든 txt 파일 읽어와
raw_corpus 리스트에 문장 단위로 저장
"""

import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 리스트에 담기
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['How does a bastard, orphan, son of a whore', 'And a Scotsman, dropped in the middle of a forgotten spot in the Caribbean by providence impoverished,', 'In squalor, grow up to be a hero and a scholar? The ten-dollar founding father without a father']


<br>

## 데이터 전처리 : 필터링

<br>

In [5]:
"""
데이터 전처리
필터링 ( 원하는 학습에 적합하도록 데이터 정제 )
1. 소문자화
2. 특수문자 제거
3. RNN 학습 위한 <start> <end> 추가
"""

import re

def preprocess_sentence(sentence):

    # 1. 소문자로 바꾸고 양쪽 공백을 삭제
    sentence = sentence.lower().strip()
  
    # 2. 아래 3단계를 거쳐 sentence를 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바꿈
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)      # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)             # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)    # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    # 3. RNN 학습에 용이하도록 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    sentence = '<start> ' + sentence + ' <end>'     
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [19]:
"""
데이터 전처리를 끝낸 말뭉치(corpus) 재생성
"""

corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue     # 공백 문장(줄) 제외
    if sentence[-1] == ":": continue    # : 로 끝나는 문장 제외
    
    preprocessed_sentence = preprocess_sentence(sentence)

    # 공백 개수로 토큰이 될 어휘 개수가 15개 넘어가는 문장 제외
    if preprocessed_sentence.count(" ") > 14: continue     

    corpus.append(preprocessed_sentence)

print(len(corpus), "\n")
corpus[:20]

156013 



['<start> how does a bastard , orphan , son of a whore <end>',
 '<start> got a lot farther by working a lot harder <end>',
 '<start> by being a lot smarter by being a self starter <end>',
 '<start> across the waves , he struggled and kept his guard up <end>',
 '<start> inside , he was longing for something to be a part of <end>',
 '<start> our man saw his future drip , dripping down the drain <end>',
 '<start> put a pencil to his temple , connected it to his brain <end>',
 '<start> took up a collection just to send him to the mainland <end>',
 '<start> get your education , don t forget from whence you came <end>',
 '<start> and the world is gonna know your name <end>',
 '<start> what s your name , man ? alexander hamilton <end>',
 '<start> my name is alexander hamilton <end>',
 '<start> and there s a million things i haven t done <end>',
 '<start> two years later , see alex and his mother bed ridden <end>',
 '<start> left him with nothin but ruined pride , something new inside voice sa

<br>

### 데이터 전처리 : 토큰화 (Tokenize)와 벡터화(Vectorize) <br><br>

__단어사전 {단어:숫자}__ <br>
컴퓨터는 숫자로 의사소통을 합니다. 우리가 쓰는 언어(단어)를 숫자로 변환하여 컴퓨터에게 넘겨주어야 합니다. <br><br>

이를 위해 {단어:숫자} 형태의 __단어사전__이 필요합니다. <br>
단어사전은 __토큰화(tokenize)__를 통해 만듭니다. <br>
이 때, 어휘(단어)를 숫자로 변환시켜주는 __벡터화(vectorize)__과정이 필요합니다. <br>
이렇게 숫자로 변환된 데이터를 __텐서(tensor)__라고 합니다. <br><br>

__tf.keras.preprocessing.text.Tokenizer 패키지__ <br>
데이터의 토큰화, 벡터화를 한번에 해주는 tensorflow.keras의 전처리 라이브러리 입니다. <br>
정제된 데이터를 토큰화하고, 단어 사전을 만들어주며, <br>
데이터를 숫자로 변환하는 벡터화를 통해 텐서를 생성합니다.

<br>

In [7]:
"""
데이터 전처리
tf.keras.preprocessing.text.Tokenizer 패키지를 통한 토큰화(tokenize), 벡터화(vetorize)

토큰화 : preprocessing.text.Tokenizer.fit_on_texts()
벡터화 : preprocessing.text.Tokenizer.texts_to_sequences()
패딩 : preprocessing.sequence.pad_sequences()
"""

def tokenize(corpus):

    # 텐서플로우의 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,     # 단어장의 크기 (전체 단어의 개수) 
        filters=' ',        # 별도로 전처리 로직을 추가할 수 있습니다. (이번에는 사용하지 않겠습니다)
        oov_token="<unk>"   # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )

    # 위에서 정제한 corpus에 대해 Tokenizer.fit_on_texts()를 통해 단어사전 생성
    tokenizer.fit_on_texts(corpus)   

    # Tokenizer.texts_to_sequences()를 통해 모델에 입력할 벡터(tensor) 데이터셋을 생성
    tensor = tokenizer.texts_to_sequences(corpus)   # 위에서 생성한 단어사전으로부터 corpus를 해석해 Tensor로 변환

    # padding 메소드 : 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위해 패딩
    # maxlen의 디폴트값은 None : 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor, "\n\n", tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[  2  79 622 ...   3   0   0]
 [  2  37   9 ...   0   0   0]
 [  2 115 573 ...   0   0   0]
 ...
 [  2   6 225 ...   0   0   0]
 [  2   4  61 ...   0   0   0]
 [  2   6 225 ...   0   0   0]] 



In [15]:
# 데이터 전처리 과정에서 토큰 개수가 15개 맞는지 확인
# tensor 변수의 데이터 형태 확인
print(len(tensor[0]))
print(tensor.shape)

15
(156013, 15)


In [48]:
"""
생성된 텐서 데이터 확인 (3번째 행, 10번째까지만 출력)
"""

print(tensor[:3, :10])

[[   2   79  622    9 4486    5    1    5  646   20]
 [   2   37    9  441 3069  115 1141    9  441 1046]
 [   2  115  573    9  441 6826  115  573    9 1163]]


In [47]:
"""
Tokenizer에 구축된 단어 사전의 인덱스 확인
"""

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


<br>

### Source sentecne 와 Target sentence 분리

<br>

In [21]:
"""
RNN 학습을 위한 입력데이터 생성
벡터화된 텐서데이터로부터 Source sentence 와 Tartget sentence 로 분리
"""

# tensor에서 마지막 토큰을 잘라내서 source sentence를 생성합니다. ( 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다 )
src_input = tensor[:, :-1]    
# tensor에서 <start>를 잘라내서 target sentence를 생성합니다.  
tgt_input = tensor[:, 1:]       

print(src_input[0])
print(tgt_input[0])

[    2    79   622     9  4486     5 10323     5   646    20     9  3397
     3     0]
[   79   622     9  4486     5 10323     5   646    20     9  3397     3
     0     0]


In [22]:
"""
sklearn으로 train셋과 test셋 분리 (모델 학습용)
"""

from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                            tgt_input, 
                                                            test_size=0.2,
                                                            shuffle=True)
print(enc_train)
print("Source Train : ", enc_train.shape)
print("Target Train : ", dec_train.shape)

[[  2  33 261 ...   0   0   0]
 [  2  47   5 ...   0   0   0]
 [  2  79 300 ...   0   0   0]
 ...
 [  2   7  56 ...  76 457 136]
 [  2  44  17 ...   0   0   0]
 [  2   4  34 ...   0   0   0]]
Source Train :  (124810, 14)
Target Train :  (124810, 14)


<br>

### 데이터셋 객체 생성 <br><br>

그동안 우리는 model.fit(x_train, y_train, …) 형태로 Numpy Array 데이터셋을 생성하여 model에 제공하는 형태의 학습을 많이 진행해 왔습니다. <br>
그러나 텐서플로우를 활용할 경우 텐서로 생성된 데이터를 이용해 tf.data.Dataset객체를 생성하는 방법을 흔히 사용합니다. <br><br>

__tf.data.Dataset객체__ <br>
tf.data.Dataset객체는 텐서플로우에서 사용할 경우 데이터 입력 파이프라인을 통한 속도 개선 및 각종 편의기능을 제공하므로 꼭 사용법을 알아 두시기를 권합니다. 우리는 이미 데이터셋을 텐서 형태로 생성해 두었으므로, tf.data.Dataset.from_tensor_slices() 메소드를 이용해 tf.data.Dataset객체를 생성할 것입니다.

<br>

In [24]:
"""
tf.data.Dataset 으로 학습을 위한 데이터셋 생성
학습을 위한 하이퍼파라미터 설정
"""

BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

<br>

## 모델 학습

<br>

<br>

### 모델 구성 <br><br>

우리가 만들 모델은 tf.keras.Model을 Subclassing하는 방식으로 만들 것입니다. 우리가 만들 모델에는 1개의 Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있습니다. <br><br>

<br>

In [25]:
"""
모델 생성
enbedding layer 와 hidden layer 활용
"""

class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [26]:
"""
모델 확인

model은 아직 제대로 build되지 않았습니다.
model.compile()을 호출한 적도 없고, 아직 model의 입력 텐서가 무엇인지 제대로 지정해 주지도 않았기 때문입니다.

그런 경우 아래와 같이 model에 데이터를 아주 조금 태워 보는 것도 방법입니다.)
model의 input shape가 결정되면서 model.build()가 자동으로 호출됩니다.
"""

for src_sample, tgt_sample in dataset.take(1): break    # 하나의 배치만 데이터를 가져와 모델에 통과(?)
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[ 1.55865710e-04,  5.37781489e-05,  1.24139740e-04, ...,
         -9.72222188e-06, -9.23581829e-05, -1.56753813e-04],
        [ 4.14280774e-04,  1.05145438e-04,  2.60636007e-04, ...,
          2.22485560e-05, -4.36053990e-04, -1.82532487e-04],
        [ 4.48071049e-04, -5.22301489e-05,  2.59812165e-04, ...,
          2.49494100e-04, -6.51614740e-04,  1.44018923e-04],
        ...,
        [-1.34621761e-04, -6.27547270e-04,  6.45147287e-04, ...,
         -3.60006758e-04, -4.43128956e-04,  1.71524985e-03],
        [-3.85105785e-04, -7.15317205e-04,  6.17600221e-04, ...,
         -3.78073019e-04, -3.26665468e-04,  1.71787036e-03],
        [-5.00671507e-04, -5.24935022e-04,  5.40392590e-04, ...,
         -4.42377874e-04, -2.02564246e-04,  1.92892039e-03]],

       [[ 1.55865710e-04,  5.37781489e-05,  1.24139740e-04, ...,
         -9.72222188e-06, -9.23581829e-05, -1.56753813e-04],
        [ 1.18166237e-04,  3.38870304e-04,  8

In [27]:
"""
모델 구성 출력
"""

model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


<br>

### 모델 학습 <br><br>

tensorflow를 통한 모델학습은 model.compile() 후에 model.fit()을 진행해야 합니다.

<br>

In [28]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)       # 10 epoch 안에 val_loss 2.2 수준으로 만들기

Epoch 1/10
609/609 [==============================] - 82s 134ms/step - loss: 3.4220
Epoch 2/10
609/609 [==============================] - 85s 139ms/step - loss: 2.9546
Epoch 3/10
609/609 [==============================] - 85s 139ms/step - loss: 2.7811
Epoch 4/10
609/609 [==============================] - 83s 137ms/step - loss: 2.6520
Epoch 5/10
609/609 [==============================] - 84s 138ms/step - loss: 2.5422
Epoch 6/10
609/609 [==============================] - 85s 140ms/step - loss: 2.4430
Epoch 7/10
609/609 [==============================] - 86s 142ms/step - loss: 2.3518
Epoch 8/10
609/609 [==============================] - 86s 142ms/step - loss: 2.2657
Epoch 9/10
609/609 [==============================] - 86s 142ms/step - loss: 2.1852
Epoch 10/10
609/609 [==============================] - 84s 138ms/step - loss: 2.1085


<br>

### 모델 평가하기

<br>

In [29]:
"""
작문 함수 작성
작문 모델을 평가하는 가장 확실한 방법은 작문을 시켜보고 직접 평가하는 겁니다. 
아래 generate_text 함수는 모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행하게 합니다.
"""

def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [31]:
"""
문장생성 함수로 문장 생성
"""

generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '